# Hyperparameter Tuning with Grid Search

In [ ]:
import csv
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report, accuracy_score

TEST_SIZE = 0.2

def load_data(filename):
    evidence = []
    labels = []

    month_index = {
        "Jan": 0, "Feb": 1, "Mar": 2, "Apr": 3, "May": 4, "June": 5,
        "Jul": 6, "Aug": 7, "Sep": 8, "Oct": 9, "Nov": 10, "Dec": 11
    }

    with open(filename) as f:
        reader = csv.DictReader(f)
        for row in reader:
            evidence.append([
                int(row["Administrative"]),
                float(row["Administrative_Duration"]),
                int(row["Informational"]),
                float(row["Informational_Duration"]),
                int(row["ProductRelated"]),
                float(row["ProductRelated_Duration"]),
                float(row["BounceRates"]),
                float(row["ExitRates"]),
                float(row["PageValues"]),
                float(row["SpecialDay"]),
                month_index[row["Month"]],
                int(row["OperatingSystems"]),
                int(row["Browser"]),
                int(row["Region"]),
                int(row["TrafficType"]),
                1 if row["VisitorType"] == "Returning_Visitor" else 0,
                1 if row["Weekend"] == "TRUE" else 0,
            ])
            labels.append(1 if row["Revenue"] == "TRUE" else 0)

    return evidence, labels

def evaluate(labels, predictions):
    t_positive = float(0)
    t_negative = float(0)
    sensitivity = float(0)
    specificity = float(0)

    for label, prediction in zip(labels, predictions):
        if label == 0:
            t_negative += 1
            if label == prediction:
                specificity += 1

        if label == 1:
            t_positive += 1
            if label == prediction:
                sensitivity += 1

    sensitivity /= t_positive
    specificity /= t_negative

    return sensitivity, specificity

# Replace 'Data/activity_data.csv' with the path to your CSV file
filename = 'activity_data.csv'

evidence, labels = load_data(filename)

X_train, X_test, y_train, y_test = train_test_split(evidence, labels, test_size=TEST_SIZE, random_state=42)

# Define the parameter grid for RBF and Polynomial kernels
param_grid = [
    {
        'svc__kernel': ['rbf'],
        'svc__C': [0.1, 1, 10, 100],
        'svc__gamma': [1, 0.1, 0.01, 0.001]
    },
    {
        'svc__kernel': ['poly'],
        'svc__C': [0.1, 1, 10, 100],
        'svc__degree': [2, 3, 4],
        'svc__gamma': ['scale', 'auto']
    }
]

# Create a pipeline with StandardScaler and SVC
pipeline = make_pipeline(StandardScaler(), SVC())

# Perform grid search with cross-validation
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Best model after grid search
best_model = grid_search.best_estimator_

# Make predictions with the best model
predictions = best_model.predict(X_test)

# Evaluate the model
sensitivity, specificity = evaluate(y_test, predictions)
print(f"Best Parameters: {grid_search.best_params_}")
print(f"Correct: {(y_test == predictions).sum()}")
print(f"Incorrect: {(y_test != predictions).sum()}")
print(f"True Positive Rate: {100 * sensitivity:.2f}%")
print(f"True Negative Rate: {100 * specificity:.2f}%")

print("\nClassification Report\n")
print(classification_report(y_test, predictions))
print("Accuracy: ", accuracy_score(y_test, predictions) * 100)

# Hyperparameter Tuning with Randomised Search

In [ ]:
import csv
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report, accuracy_score
from scipy.stats import uniform, randint

TEST_SIZE = 0.2

def load_data(filename):
    evidence = []
    labels = []

    month_index = {
        "Jan": 0, "Feb": 1, "Mar": 2, "Apr": 3, "May": 4, "June": 5,
        "Jul": 6, "Aug": 7, "Sep": 8, "Oct": 9, "Nov": 10, "Dec": 11
    }

    with open(filename) as f:
        reader = csv.DictReader(f)
        for row in reader:
            evidence.append([
                int(row["Administrative"]),
                float(row["Administrative_Duration"]),
                int(row["Informational"]),
                float(row["Informational_Duration"]),
                int(row["ProductRelated"]),
                float(row["ProductRelated_Duration"]),
                float(row["BounceRates"]),
                float(row["ExitRates"]),
                float(row["PageValues"]),
                float(row["SpecialDay"]),
                month_index[row["Month"]],
                int(row["OperatingSystems"]),
                int(row["Browser"]),
                int(row["Region"]),
                int(row["TrafficType"]),
                1 if row["VisitorType"] == "Returning_Visitor" else 0,
                1 if row["Weekend"] == "TRUE" else 0,
            ])
            labels.append(1 if row["Revenue"] == "TRUE" else 0)

    return evidence, labels

def evaluate(labels, predictions):
    t_positive = float(0)
    t_negative = float(0)
    sensitivity = float(0)
    specificity = float(0)

    for label, prediction in zip(labels, predictions):
        if label == 0:
            t_negative += 1
            if label == prediction:
                specificity += 1

        if label == 1:
            t_positive += 1
            if label == prediction:
                sensitivity += 1

    sensitivity /= t_positive
    specificity /= t_negative

    return sensitivity, specificity


# Replace 'Data/activity_data.csv' with the path to your CSV file
filename = 'activity_data.csv'

evidence, labels = load_data(filename)

X_train, X_test, y_train, y_test = train_test_split(evidence, labels, test_size=TEST_SIZE, random_state=42)

# Define the parameter distribution for RBF and Polynomial kernels
param_dist = {
    'svc__kernel': ['rbf', 'poly'],
    'svc__C': uniform(0.1, 100),
    'svc__gamma': uniform(0.001, 0.1),
    'svc__degree': randint(2, 5)  # Only relevant for 'poly' kernel
}

# Create a pipeline with StandardScaler and SVC
pipeline = make_pipeline(StandardScaler(), SVC())

# Perform randomized search with cross-validation
random_search = RandomizedSearchCV(pipeline, param_distributions=param_dist, n_iter=50, cv=5, scoring='accuracy', random_state=42, n_jobs=-1)
random_search.fit(X_train, y_train)

# Best model after randomized search
best_model = random_search.best_estimator_

# Make predictions with the best model
predictions = best_model.predict(X_test)

# Evaluate the model
sensitivity, specificity = evaluate(y_test, predictions)
print(f"Best Parameters: {random_search.best_params_}")
print(f"Correct: {(y_test == predictions).sum()}")
print(f"Incorrect: {(y_test != predictions).sum()}")
print(f"True Positive Rate: {100 * sensitivity:.2f}%")
print(f"True Negative Rate: {100 * specificity:.2f}%")

print("\nClassification Report\n")
print(classification_report(y_test, predictions))
print("Accuracy: ", accuracy_score(y_test, predictions) * 100)

# Randomised

In [ ]:
import csv
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report, accuracy_score
from sklearn.decomposition import PCA

TEST_SIZE = 0.2

def load_data(filename):
    evidence = []
    labels = []

    month_index = {
        "Jan": 0, "Feb": 1, "Mar": 2, "Apr": 3, "May": 4, "June": 5,
        "Jul": 6, "Aug": 7, "Sep": 8, "Oct": 9, "Nov": 10, "Dec": 11
    }

    with open(filename) as f:
        reader = csv.DictReader(f)
        for row in reader:
            evidence.append([
                int(row["Administrative"]),
                float(row["Administrative_Duration"]),
                int(row["Informational"]),
                float(row["Informational_Duration"]),
                int(row["ProductRelated"]),
                float(row["ProductRelated_Duration"]),
                float(row["BounceRates"]),
                float(row["ExitRates"]),
                float(row["PageValues"]),
                float(row["SpecialDay"]),
                month_index[row["Month"]],
                int(row["OperatingSystems"]),
                int(row["Browser"]),
                int(row["Region"]),
                int(row["TrafficType"]),
                1 if row["VisitorType"] == "Returning_Visitor" else 0,
                1 if row["Weekend"] == "TRUE" else 0,
            ])
            labels.append(1 if row["Revenue"] == "TRUE" else 0)

    return evidence, labels

def evaluate(labels, predictions):
    t_positive = float(0)
    t_negative = float(0)
    sensitivity = float(0)
    specificity = float(0)

    for label, prediction in zip(labels, predictions):
        if label == 0:
            t_negative += 1
            if label == prediction:
                specificity += 1

        if label == 1:
            t_positive += 1
            if label == prediction:
                sensitivity += 1

    sensitivity /= t_positive
    specificity /= t_negative

    return sensitivity, specificity

filename = 'activity_data.csv'

evidence, labels = load_data(filename)

X_train, X_test, y_train, y_test = train_test_split(evidence, labels, test_size=TEST_SIZE, random_state=42)

param_distributions = [
    {
        'svc__kernel': ['rbf'],
        'svc__C': [0.1, 1, 10, 100],
        'svc__gamma': [1, 0.1, 0.01, 0.001]
    },
    {
        'svc__kernel': ['poly'],
        'svc__C': [0.1, 1, 10, 100],
        'svc__degree': [2, 3, 4],
        'svc__gamma': ['scale', 'auto']
    }
]

pipeline = make_pipeline(StandardScaler(), PCA(n_components=10), SVC())

random_search = RandomizedSearchCV(pipeline, param_distributions, n_iter=10, cv=5, scoring='accuracy', n_jobs=-1)
random_search.fit(X_train, y_train)

best_model = random_search.best_estimator_
predictions = best_model.predict(X_test)

print(f"Accuracy: {accuracy_score(y_test, predictions)}")
print(f"Classification Report:\n{classification_report(y_test, predictions)}")
